In [0]:
from pyspark.sql import SparkSession # step 1 import spark session

In [0]:
#dbfs:/FileStore/shared_uploads/n.n_sultan@hotmail.com/sellers/part_00000_54f48a9e_d67c_4c29_aed7_1b5f6f117c12_c000_snappy.parquet
spark = SparkSession.builder.appName('problems').getOrCreate()

In [0]:
df_sellers = spark.read.parquet('dbfs:/FileStore/shared_uploads/n.n_sultan@hotmail.com/sellers/')


In [0]:
df_sales = spark.read.parquet('dbfs:/FileStore/shared_uploads/n.n_sultan@hotmail.com/sales/')

In [0]:
#dbfs:/FileStore/shared_uploads/n.n_sultan@hotmail.com/products/
df_products = spark.read.parquet('dbfs:/FileStore/shared_uploads/n.n_sultan@hotmail.com/products/')

In [0]:
print(df_products.count())
print(df_sales.count())
print(df_sellers.count())

75000000
939932
10

In [0]:
from pyspark.sql.functions import countDistinct, col, count, avg

In [0]:
df_sales.select(countDistinct(df_sales['product_id'])).show()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-3645135835077234> in <module> 
 ----> 1 df_sales . select ( countDistinct ( df_sales [ 'product_id' ] ) ) . show ( ) 

 /databricks/spark/python/pyspark/sql/dataframe.py in show (self, n, truncate, vertical) 
 439 """
 440 if isinstance ( truncate , bool ) and truncate : 
 --> 441 print ( self . _jdf . showString ( n , 20 , vertical ) ) 
 442 else : 
 443 print ( self . _jdf . showString ( n , int ( truncate ) , vertical ) ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1303 answer = self . gateway_client . send_command ( command ) 
 1304 return_value = get_return_value(
 -> 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 
 1307 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 125 def deco ( * a , ** kw ) : 
 126 try : 
 --> 127 return f ( * a , ** kw ) 
 128 except py4j . protocol . Py4JJavaError as e : 
 129 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o448.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 5 in stage 9.0 failed 1 times, most recent failure: Lost task 5.0 in stage 9.0 (TID 31, ip-10-172-232-248.us-west-2.compute.internal, executor driver): com.databricks.sql.io.FileReadException: Error while reading file dbfs:REDACTED_LOCAL_PART@hotmail.com/sales/part_00037_e651a798_938c_4056_96ac_b29d9c3b497e_c000_snappy.parquet.
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1$$anon$2.logFileNameAndThrow(FileScanRDD.scala:343)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1$$anon$2.getNext(FileScanRDD.scala:322)
	at org.apache.spark.util.NextIterator.hasNext(NextIterator.scala:73)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.$anonfun$prepareNextFile$1(FileScanRDD.scala:475)
	at scala.concurrent.Future$.$anonfun$apply$1(Future.scala:659)
	at scala.util.Success.$anonfun$map$1(Try.scala:255)
	at scala.util.Success.map(Try.scala:213)
	at scala.concurrent.Future.$anonfun$map$1(Future.scala:292)
	at scala.concurrent.impl.Promise.liftedTree1$1(Promise.scala:33)
	at scala.concurrent.impl.Promise.$anonfun$transform$1(Promise.scala:33)
	at scala.concurrent.impl.CallbackRunnable.run(Promise.scala:64)
	at org.apache.spark.util.threads.SparkThreadLocalCapturingRunnable.$anonfun$run$1(SparkThreadLocalForwardingThreadPoolExecutor.scala:104)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.util.threads.SparkThreadLocalCapturingHelper.runWithCaptured(SparkThreadLocalForwardingThreadPoolExecutor.scala:68)
	at org.apache.spark.util.threads.SparkThreadLocalCapturingHelper.runWithCaptured$(SparkThreadLocalForwardingThreadPoolExecutor.scala:54)
	at org.apache.spark.util.threads.SparkThreadLocalCapturingRunnable.runWithCaptured(SparkThreadLocalForwardingThreadPoolExecutor.scala:101)
	at org.apache.spark.util.threads.SparkThreadLocalCapturingRunnable.run(SparkThreadLocalForwardingThreadPoolExecutor.scala:104)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.io.IOException: can not read class org.apache.parquet.format.PageHeader: don't know what type: 15
	at org.apache.parquet.format.Util.read(Util.java:216)
	at org.apache.parquet.format.Util.readPageHeader(Util.java:65)
	at org.apache.parquet.hadoop.ParquetFileReader$WorkaroundChunk.re

In [0]:
df_sales.agg(countDistinct(col('product_id'))).show()

In [0]:
from pyspark.sql import Row
from pyspark.sql.types import IntegerType

In [0]:
from pyspark.sql.functions import *

In [0]:
df_sales = df_sales.withColumn('Salted_join_key_b', when(df_sales['product_id'].isin(replicated_products), concat(df_sales['product_id'], lit('-'), round(rand() * (replication_facotr-1),0).cast(IntegerType()))).otherwise(df_sales['product_id']))

In [0]:
df_sales.show()

+--------+----------+---------+----------+---------------+--------------------+-----------------+
order_id|product_id|seller_id| date|num_pieces_sold| bill_raw_text|Salted_join_key_b|
+--------+----------+---------+----------+---------------+--------------------+-----------------+
 475442| 59216308| 4|2020-07-07| 11|bdfihbvlcxeqioqgm...| 59216308|
 475460| 64349211| 9|2020-07-05| 93|shwfatjysteuxvjfl...| 64349211|
 475745| 6147551| 6|2020-07-07| 33|czhqinijvbhlsljac...| 6147551|
 475748| 18524928| 1|2020-07-09| 3|nfjxfasnrnivfxfdt...| 18524928|
 475765| 44920059| 6|2020-07-08| 77|kqhmsiqtofuowcwfv...| 44920059|
 476233| 53991698| 4|2020-07-10| 10|gfsgwwnduxpmevgxc...| 53991698|
 476247| 9083087| 8|2020-07-02| 45|qbvwimpyblobdeeas...| 9083087|
 476342| 56844041| 1|2020-07-07| 28|jentbiirsqyvzmtwu...| 56844041|
 476378| 4583362| 5|2020-07-07| 44|kntxdtphezzxiencr...| 4583362|
 476436| 3226992| 7|2020-07-09| 66|epdfkexlllyngarnn...| 3226992|
 476605| 16478518| 3|2020-07-02| 67|sowhqsofazefdykou...| 16478518|
 476613| 26742725| 1|2020-07-02| 86|kebqcojololstaQsf...| 26742725|
 476618| 67843613| 6|2020-07-06| 39|fnszornnjqbunsbdk...| 67843613|
 476904| 52164566| 3|2020-07-08| 1|qxvyxwulfbacmjscl...| 52164566|
 476913| 43047781| 7|2020-07-03| 43|arvjmninostqhxaov...| 43047781|
 477268| 64032736| 6|2020-07-10| 44|dzoywhjjsdeqenrvt...| 64032736|
 477388| 31763266| 5|2020-07-06| 89|ekyqcbazdleiqpanj...| 31763266|
 477481| 29651493| 4|2020-07-04| 18|fkoysmkjxxvweblog...| 29651493|
 477569| 33046297| 2|2020-07-06| 24|zkywltcuzhomjwcwd...| 33046297|
 477714| 42874375| 1|2020-07-01| 21|lvosqtykdjangBdnw...| 42874375|
+--------+----------+---------+----------+---------------+--------------------+-----------------+
only showing top 20 rows

In [0]:
df_sales.filter(df_sales['product_id'] == '18182299').show()

+--------+----------+---------+----------+---------------+--------------------+-----------------+
order_id|product_id|seller_id| date|num_pieces_sold| bill_raw_text|Salted_join_key_b|
+--------+----------+---------+----------+---------------+--------------------+-----------------+
 6989092| 18182299| 4|2020-07-03| 31|mioxdhptbyuzsvejk...| 18182299|
13477744| 18182299| 7|2020-07-05| 15|lvfcuxmuqoyvbzIll...| 18182299|
18489905| 18182299| 6|2020-07-06| 67|wmaaaxqtzpzphxgyn...| 18182299|
+--------+----------+---------+----------+---------------+--------------------+-----------------+

In [0]:
df_sellers.show()

+---------+-----------+------------+
seller_id|seller_name|daily_target|
+---------+-----------+------------+
 0| seller_0| 2500000|
 1| seller_1| 257237|
 2| seller_2| 754188|
 3| seller_3| 310462|
 4| seller_4| 1532808|
 5| seller_5| 1199693|
 6| seller_6| 1055915|
 7| seller_7| 1946998|
 8| seller_8| 547320|
 9| seller_9| 1318051|
+---------+-----------+------------+

In [0]:
df_sales_sellers_ratio = df_sales.join(broadcast(df_sellers), df_sellers['seller_id'] == df_sales['seller_id'], 'inner').withColumn('ratio', df_sales['num_pieces_sold']/df_sellers['daily_target']).groupby(df_sellers['seller_id']).agg(avg(col('ratio'))).orderBy(col('seller_id'))

In [0]:
df_sales.show()

+--------+----------+---------+----------+---------------+--------------------+-----------------+
order_id|product_id|seller_id| date|num_pieces_sold| bill_raw_text|Salted_join_key_b|
+--------+----------+---------+----------+---------------+--------------------+-----------------+
 475442| 59216308| 4|2020-07-07| 11|bdfihbvlcxeqioqgm...| 59216308|
 475460| 64349211| 9|2020-07-05| 93|shwfatjysteuxvjfl...| 64349211|
 475745| 6147551| 6|2020-07-07| 33|czhqinijvbhlsljac...| 6147551|
 475748| 18524928| 1|2020-07-09| 3|nfjxfasnrnivfxfdt...| 18524928|
 475765| 44920059| 6|2020-07-08| 77|kqhmsiqtofuowcwfv...| 44920059|
 476233| 53991698| 4|2020-07-10| 10|gfsgwwnduxpmevgxc...| 53991698|
 476247| 9083087| 8|2020-07-02| 45|qbvwimpyblobdeeas...| 9083087|
 476342| 56844041| 1|2020-07-07| 28|jentbiirsqyvzmtwu...| 56844041|
 476378| 4583362| 5|2020-07-07| 44|kntxdtphezzxiencr...| 4583362|
 476436| 3226992| 7|2020-07-09| 66|epdfkexlllyngarnn...| 3226992|
 476605| 16478518| 3|2020-07-02| 67|sowhqsofazefdykou...| 16478518|
 476613| 26742725| 1|2020-07-02| 86|kebqcojololstaQsf...| 26742725|
 476618| 67843613| 6|2020-07-06| 39|fnszornnjqbunsbdk...| 67843613|
 476904| 52164566| 3|2020-07-08| 1|qxvyxwulfbacmjscl...| 52164566|
 476913| 43047781| 7|2020-07-03| 43|arvjmninostqhxaov...| 43047781|
 477268| 64032736| 6|2020-07-10| 44|dzoywhjjsdeqenrvt...| 64032736|
 477388| 31763266| 5|2020-07-06| 89|ekyqcbazdleiqpanj...| 31763266|
 477481| 29651493| 4|2020-07-04| 18|fkoysmkjxxvweblog...| 29651493|
 477569| 33046297| 2|2020-07-06| 24|zkywltcuzhomjwcwd...| 33046297|
 477714| 42874375| 1|2020-07-01| 21|lvosqtykdjangBdnw...| 42874375|
+--------+----------+---------+----------+---------------+--------------------+-----------------+
only showing top 20 rows

In [0]:
df_sales_sellers_ratio.show()

+---------+--------------------+
seller_id| avg(ratio)|
+---------+--------------------+
 1|1.964806972235766...|
 2|6.694011384112543E-5|
 3|1.628799630191635...|
 4|3.294459121336001...|
 5|4.209736858094406E-5|
 6|4.780992199587733E-5|
 7|2.593982034749280...|
 8|9.217306560992326E-5|
 9|3.837160811881674...|
+---------+--------------------+

In [0]:
df_sales_1 = df_sales.groupBy(df_sales['seller_id'],df_sales['product_id']).agg(sum(df_sales['num_pieces_sold']).alias('num_pieces_sold'))

In [0]:
from pyspark.sql import Row, Window

In [0]:
window_desc = Window.partitionBy('product_id').orderBy(col('num_pieces_sold').desc())

In [0]:
window_asc = Window.partitionBy(col('product_id')).orderBy(col('num_pieces_sold').asc())

In [0]:
df_sales_1 = df_sales_1.withColumn('rank_asc', dense_rank().over(window_asc)).withColumn('rank_desc', dense_rank().over(window_desc))

In [0]:
df_sales_1.show()

+---------+----------+---------------+--------+---------+
seller_id|product_id|num_pieces_sold|rank_asc|rank_desc|
+---------+----------+---------------+--------+---------+
 3| 10016462| 60.0| 1| 1|
 9| 10016743| 18.0| 1| 1|
 3| 10018083| 100.0| 1| 1|
 7| 10019379| 50.0| 1| 1|
 5| 10062865| 78.0| 1| 1|
 1| 10130834| 83.0| 1| 1|
 1| 10138125| 49.0| 1| 1|
 1| 10161552| 37.0| 1| 1|
 2| 10167916| 69.0| 1| 1|
 1| 10180027| 83.0| 1| 1|
 6| 1027288| 90.0| 1| 1|
 4| 10301403| 88.0| 1| 1|
 1| 10308659| 56.0| 1| 1|
 5| 10324704| 85.0| 1| 1|
 3| 10330070| 11.0| 1| 1|
 5| 10333550| 52.0| 1| 1|
 2| 10350570| 41.0| 1| 1|
 3| 10353585| 51.0| 1| 1|
 8| 10382040| 90.0| 1| 1|
 9| 10391484| 61.0| 1| 1|
+---------+----------+---------------+--------+---------+
only showing top 20 rows

In [0]:
single_seller = df_sales_1.where(col('rank_asc') == col('rank_desc')).select(
    col("product_id").alias("single_seller_product_id"), col("seller_id").alias("single_seller_seller_id"),col('rank_desc'),
    lit("Only seller or multiple sellers with the same results").alias("type")
)

In [0]:
single_seller.show()

+------------------------+-----------------------+---------+--------------------+
single_seller_product_id|single_seller_seller_id|rank_desc| type|
+------------------------+-----------------------+---------+--------------------+
 10016462| 3| 1|Only seller or mu...|
 10016743| 9| 1|Only seller or mu...|
 10018083| 3| 1|Only seller or mu...|
 10019379| 7| 1|Only seller or mu...|
 10062865| 5| 1|Only seller or mu...|
 10130834| 1| 1|Only seller or mu...|
 10138125| 1| 1|Only seller or mu...|
 10161552| 1| 1|Only seller or mu...|
 10167916| 2| 1|Only seller or mu...|
 10180027| 1| 1|Only seller or mu...|
 1027288| 6| 1|Only seller or mu...|
 10301403| 4| 1|Only seller or mu...|
 10308659| 1| 1|Only seller or mu...|
 10324704| 5| 1|Only seller or mu...|
 10330070| 3| 1|Only seller or mu...|
 10333550| 5| 1|Only seller or mu...|
 10350570| 2| 1|Only seller or mu...|
 10353585| 3| 1|Only seller or mu...|
 10382040| 8| 1|Only seller or mu...|
 10391484| 9| 1|Only seller or mu...|
+------------------------+-----------------------+---------+--------------------+
only showing top 20 rows

In [0]:
second_seller = df_sales_1.where(col('rank_desc') == 2).select(col('product_id').alias('second_seller_product_id'), col('seller_id').alias('second_seller_seller_id'), col('rank_desc'),lit('Second top Seller').alias('type'))

In [0]:
second_seller.show()

+------------------------+-----------------------+---------+-----------------+
second_seller_product_id|second_seller_seller_id|rank_desc| type|
+------------------------+-----------------------+---------+-----------------+
 12385827| 5| 2|Second top Seller|
 17093664| 6| 2|Second top Seller|
 17250450| 5| 2|Second top Seller|
 18424380| 6| 2|Second top Seller|
 22834213| 2| 2|Second top Seller|
 23067656| 1| 2|Second top Seller|
 27539090| 5| 2|Second top Seller|
 28387702| 2| 2|Second top Seller|
 28908793| 3| 2|Second top Seller|
 29727816| 7| 2|Second top Seller|
 40999836| 5| 2|Second top Seller|
 43037162| 4| 2|Second top Seller|
 43843468| 8| 2|Second top Seller|
 49966300| 7| 2|Second top Seller|
 51535842| 5| 2|Second top Seller|
 52036011| 6| 2|Second top Seller|
 5211602| 6| 2|Second top Seller|
 52209540| 6| 2|Second top Seller|
 52807839| 5| 2|Second top Seller|
 53410222| 1| 2|Second top Seller|
+------------------------+-----------------------+---------+-----------------+
only showing top 20 rows

In [0]:
least_seller = df_sales_1.where(col("rank_asc") == 1).select(
    col("product_id"), col("seller_id"),
    lit("Least Seller").alias("type")
).join(single_seller, (df_sales_1["seller_id"] == single_seller["single_seller_seller_id"]) & (
        df_sales_1["product_id"] == single_seller["single_seller_product_id"]), "left_anti"). \
    join(second_seller, (df_sales_1["seller_id"] == second_seller["second_seller_seller_id"]) & (
        df_sales_1["product_id"] == second_seller["second_seller_product_id"]), "left_anti")

In [0]:
least_seller.show()

+----------+---------+------------+
product_id|seller_id| type|
+----------+---------+------------+
 19986717| 1|Least Seller|
 40496308| 5|Least Seller|
 52606213| 7|Least Seller|
 14542470| 5|Least Seller|
 28592106| 5|Least Seller|
 17944574| 8|Least Seller|
 61475460| 7|Least Seller|
 3534470| 3|Least Seller|
 35669461| 4|Least Seller|
 32602520| 9|Least Seller|
 72017876| 1|Least Seller|
 67723231| 5|Least Seller|
 56011040| 5|Least Seller|
 34681047| 5|Least Seller|
 57735075| 9|Least Seller|
 18182299| 7|Least Seller|
 69790381| 5|Least Seller|
 10978356| 7|Least Seller|
 20774718| 9|Least Seller|
 36269838| 8|Least Seller|
+----------+---------+------------+

In [0]:
union_table = least_seller.select(
    col("product_id"),
    col("seller_id"),
    col("type")
).union(second_seller.select(
    col("second_seller_product_id").alias("product_id"),
    col("second_seller_seller_id").alias("seller_id"),
    col("type")
)).union(single_seller.select(
    col("single_seller_product_id").alias("product_id"),
    col("single_seller_seller_id").alias("seller_id"),
    col("type")
))

In [0]:
union_table.show()

+----------+---------+------------+
product_id|seller_id| type|
+----------+---------+------------+
 19986717| 1|Least Seller|
 40496308| 5|Least Seller|
 52606213| 7|Least Seller|
 14542470| 5|Least Seller|
 28592106| 5|Least Seller|
 17944574| 8|Least Seller|
 61475460| 7|Least Seller|
 3534470| 3|Least Seller|
 35669461| 4|Least Seller|
 32602520| 9|Least Seller|
 72017876| 1|Least Seller|
 67723231| 5|Least Seller|
 56011040| 5|Least Seller|
 34681047| 5|Least Seller|
 57735075| 9|Least Seller|
 18182299| 7|Least Seller|
 69790381| 5|Least Seller|
 10978356| 7|Least Seller|
 20774718| 9|Least Seller|
 36269838| 8|Least Seller|
+----------+---------+------------+
only showing top 20 rows